In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,513170
1,Huelva,Confirmados PDIA,60668
2,Huelva,Confirmados PDIA 14 días,7524
3,Huelva,Tasa PDIA 14 días,"1466,1807977863086"
4,Huelva,Confirmados PDIA 7 días,3375
5,Huelva,Tasa PDIA 7 dias,"657,6767932653896"
6,Huelva,Total Confirmados,60883
7,Huelva,Curados,48180
8,Huelva,Fallecidos,427


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  60668.0


/tmp/ipykernel_176641/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  17512.0


/tmp/ipykernel_176641/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_176641/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_176641/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_176641/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 123 personas en los últimos 7 días 

Un positivo PCR cada 59 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,60668.0,7524.0,3375.0,513170.0,657.676793,1466.180798,1875.0
Huelva-Costa,35703.0,4285.0,1991.0,289548.0,687.623468,1479.892798,1199.0
Huelva (capital),17512.0,2438.0,1166.0,143837.0,810.639821,1694.974172,684.0
Condado-Campiña,18096.0,1906.0,829.0,156231.0,530.624524,1219.988351,467.0
Sierra de Huelva-Andévalo Central,6419.0,1321.0,550.0,67391.0,816.132718,1960.202401,205.0
Cartaya,2700.0,274.0,154.0,20083.0,766.817707,1364.337997,95.0
Aljaraque,2327.0,322.0,130.0,21474.0,605.383254,1499.487753,87.0
Ayamonte,2673.0,289.0,119.0,21104.0,563.874147,1369.408643,80.0
Lepe,3531.0,278.0,135.0,27880.0,484.218077,997.130560,77.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Hinojales,17.0,15.0,15.0,330.0,4545.454545,4545.454545,7.0
Rosal de la Frontera,152.0,66.0,47.0,1697.0,2769.593400,3889.216264,22.0
Villanueva de las Cruces,41.0,14.0,9.0,387.0,2325.581395,3617.571059,6.0
Campofrío,88.0,25.0,6.0,713.0,841.514727,3506.311360,0.0
San Silvestre de Guzmán,79.0,20.0,4.0,614.0,651.465798,3257.328990,0.0
Galaroza,113.0,43.0,8.0,1382.0,578.871201,3111.432706,3.0
Jabugo,158.0,63.0,26.0,2260.0,1150.442478,2787.610619,11.0
Valverde del Camino,1437.0,342.0,130.0,12750.0,1019.607843,2682.352941,35.0
Aroche,227.0,76.0,32.0,3054.0,1047.806156,2488.539620,12.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cumbres Mayores,219.0,22.0,2.0,1749.0,114.351058,1257.861635,2.0,0.090909
Alájar,51.0,8.0,1.0,759.0,131.752306,1054.018445,1.0,0.125000
Puerto Moral,27.0,6.0,1.0,278.0,359.712230,2158.273381,1.0,0.166667
Galaroza,113.0,43.0,8.0,1382.0,578.871201,3111.432706,3.0,0.186047
Escacena del Campo,309.0,46.0,9.0,2287.0,393.528640,2011.368605,4.0,0.195652
San Silvestre de Guzmán,79.0,20.0,4.0,614.0,651.465798,3257.328990,0.0,0.200000
Villarrasa,273.0,20.0,4.0,2211.0,180.913614,904.568069,3.0,0.200000
Campofrío,88.0,25.0,6.0,713.0,841.514727,3506.311360,0.0,0.240000
Cumbres de San Bartolomé,17.0,4.0,1.0,376.0,265.957447,1063.829787,0.0,0.250000
